In [ ]:
# UNCOMMENT IF USING GOOGLE-DRIVE:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Colab Notebooks/COMP34812/'
%cd /content/drive/MyDrive/Colab Notebooks/COMP34812/

# UNCOMMENT IF RUN-POD OR LOCAL
# root = ''

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/COMP34812


In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [ ]:
!pip install huggingface_hub
!pip install python-dotenv

In [77]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


class MistralNLI():
    def __init__(self, hf_token, base_model="TheBloke/Mistral-7B-v0.1-GPTQ", peft_model="jd0g/mistral-7b-nli_cot_qkv"):
        self.base_model = AutoModelForCausalLM.from_pretrained(
            base_model,
            device_map="auto",
            trust_remote_code=False,
            revision="main"
        ).cuda()
        self.tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
        self.peft_model = PeftModel.from_pretrained(self.base_model, peft_model)
        self.models = {
            "TheBloke/Mistral-7B-v0.1-GPTQ": ["Mistral-7B-NLI-v0.1", "Mistral-7B-NLI-v0.2", "mistral-7b-nli_cot_qkv", "mistral-7b-gs"]}

    def get_models(self):
        return self.models

    def generate_text(self, prompt, model_name=None) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        model = self.peft_model if model_name == "peft" else self.base_model

        response = model.generate(
            input_ids=inputs["input_ids"].to("cuda"),
            max_new_tokens=256,
            early_stopping=True
        )

        generated_text = self.tokenizer.batch_decode(response)[0]
        return generated_text

    @classmethod
    def prompt_template(cls, system: str, prompt: str = None):
        return f'''[INST] {system} \n{prompt} \n[/INST]'''

In [78]:
from dotenv import load_dotenv
import os

load_dotenv()
api_token = hf_token=os.getenv('HF_TOKEN')
mistral = MistralNLI(api_token, base_model="TheBloke/Mistral-7B-v0.1-GPTQ", peft_model="jd0g/mistral-7b-nli_cot_qkv")

In [ ]:
from service.prediction_service import predict_label, system_prompt, clean_json, validate_response

p = "I can't believe you told him that! she exclaimed."
h = "A secret was shared"

json_schema = {
    "thought_process": "<deductive/common-sense reasoning steps>",
    "label": "<0 or 1>"
}


task = f"Premise: '{p}' Hypothesis: f{h}."
prompt = mistral.prompt_template(system=system_prompt(), prompt=task)
response = mistral.generate_text(prompt, model_name="peft").split('[/INST]')[1]
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




[SOL]

{
    "thought_process": "
        Step 1: The premise indicates an exclamation of disbelief in response to a shared secret. The exclamation suggests a strong emotional reaction, which is consistent with the sharing of a secret. It is common sense that secrets are typically kept private and sharing them can be a surprise or shocking event.
        Step 2: 'fA secret was shared' can be logically inferred from the premise because the sharing of a secret is a reasonable and necessary part of the exclamation of disbelief. Thus, the hypothesis logically follows from the premise.
        Step 3: Based on the logical reasoning, common sense, and lack of contradictory facts, the hypothesis can be inferred from the premise.
        ",
    "label": 1
}

[/SOL]</s>


In [ ]:
response_json, error_message = validate_response(response, json_schema)
response_json

{'thought_process': "        step 1: the premise indicates an exclamation of disbelief in response to a shared secret. the exclamation suggests a strong emotional reaction, which is consistent with the sharing of a secret. it is common sense that secrets are typically kept private and sharing them can be a surprise or shocking event.        step 2: 'fa secret was shared' can be logically inferred from the premise because the sharing of a secret is a reasonable and necessary part of the exclamation of disbelief. thus, the hypothesis logically follows from the premise.        step 3: based on the logical reasoning, common sense, and lack of contradictory facts, the hypothesis can be inferred from the premise.        ",
 'label': 1}

In [ ]:
prediction = response_json['label']
prediction

1

In [ ]:
import pandas as pd

def get_predictions(p, h):
  task = f"Premise: '{p}' Hypothesis: f{h}."
  prompt = mistral.prompt_template(system=system_prompt(), prompt=task)
  response = mistral.generate_text(prompt, model_name="peft").split('[/INST]')[1]
  response_json, error_message = validate_response(response, json_schema)
  pprint(response_json)

  return response_json if response_json else error_message

data = pd.read_csv(root + 'data/training_data/train.csv')
sample = data.head(10)

sample['prediction_json'] = sample.apply(lambda x : get_predictions(x['premise'], x['hypothesis']), axis=1)
sample

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'label': 0,
 'thought_process': '        step 1: the premise provides information about '
                    'the history of fort charles, a military and naval '
                    'garrison that protected jamaica and the english caribbean '
                    'for 250 years. it also mentions that the fort was rebuilt '
                    'after an earthquake and that it declined after another '
                    'earthquake.        step 2: the hypothesis suggests that '
                    'fort charles was rebuilt as an amusement park for the '
                    'locals.        step 3: the hypothesis cannot be inferred '
                    'from the premise because there is no information in the '
                    'premise that supports the idea of rebuilding the fort as '
                    'an amusement park. the premise only mentions the military '
                    'and naval functions of the fort, and there is no '
                    'indication that it was ever

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


None


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

None


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


None
{'label': 1,
 'thought_process': '        step 1: the premise provides information about '
                    'the number of screenplays and film shorts worked on by an '
                    'individual over a period of 30 years.        step 2: the '
                    'hypothesis provides additional information about the '
                    'number of screenplays and film shorts worked on by the '
                    'same individual, as well as the number of television '
                    'series worked on.        step 3: the hypothesis can be '
                    'inferred from the premise because it provides additional '
                    "information about the same individual's work, without "
                    'contradicting or changing the information provided in the '
                    'premise.        '}


<ipython-input-15-3f35e2040089>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['prediction_json'] = sample.apply(lambda x : get_predictions(x['premise'], x['hypothesis']), axis=1)


,premise,hypothesis,label,prediction_json
0,"However, Fort Charles was rebuilt as a militar...",Fort Charles was rebuilt as an amusement park ...,0,{'thought_process': ' step 1: the premi...
1,Buchanan's The Democrats and Republicans have...,THe parties will never be similar.,0,"Invalid JSON object. Regenerate your response,..."
2,In order to review an acquisition that is usin...,The auditor only reviews the acquisition itsel...,0,{'thought_process': ' step 1: the premi...
3,Three young people sit outside and engage with...,There is a tin can and string telephone.,0,"Invalid JSON object. Regenerate your response,..."
4,The lucrative tin mines of Kuala Lumpur in the...,The Chinese labor was seen as less costly and ...,1,"Invalid JSON object. Regenerate your response,..."
5,"Nearby is the new, architecturally inventive S...",The Supreme Court Building was built in the ea...,1,"Invalid JSON object. Regenerate your response,..."
6,A man and a woman are sitting at a table outsi...,The man and woman are outside.,1,{'thought_process': ' step 1: the premi...
7,"Two years later, he could no longer tolerate a...","He enjoys being associated with Jews, most esp...",0,"Invalid JSON object. Regenerate your response,..."
8,.,", but because of its release being withheld, t...",1,"Invalid JSON object. Regenerate your response,..."
9,In his over 30-year career he worked on over 4...,In his over 30-year career he worked on over 4...,1,{'thought_process': ' step 1: the premi...


In [ ]:
def evaluate_metrics(df):
    true_negative = df[(df['correct'] == True) & (df['label'] == 0)].shape[0]
    true_positive = df[(df['correct'] == True) & (df['label'] == 1)].shape[0]
    false_positive = df[(df['correct'] == False) & (df['label'] == 0)].shape[0]
    false_negative = df[(df['correct'] == False) & (df['label'] == 1)].shape[0]

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    accuracy = (true_positive + true_negative) / df.shape[0]
    f1 = 2 * (precision * recall) / (precision + recall)

    return {'precision': precision, 'recall': recall, 'accuracy': accuracy, 'f1': f1}

In [82]:
from service.prediction_service import validate_response#, extract_valid_json
from llm.mistral import Mistral
from dotenv import load_dotenv
import os

load_dotenv()

def predict_label(premise: str,
                hypothesis: str,
                llm: MistralNLI,
                schema: dict,
                max_retries: int = 1,
                ) -> dict:
  """
  Predicts entailment label for a given premise-hypothesis pair using BaseLLMs.
  """
  num_of_reprompts = 0
  task = f"Premise: '{premise}' Hypothesis: {hypothesis}."
  prompt = llm.prompt_template(system=system_prompt(), prompt=task)

  print("****** INPUT ******")
  print(prompt)

  for retry in range(max_retries):

      output: str = llm.generate_text(prompt, model_name="peft")
      if '[INST]' in output:
        output = output.split('[INST]')[1]
      if '[SOL]' in output:
        output = output.split('[SOL]')[1]
      if '[ANS]' in output:
        output = output.split('[ANS]')[1]

      response_json, error_message = validate_response(output, schema)

      print("***** OUTPUT *****")
      print(output)

      if response_json:
        print("****** SUCCESS ******")
        print(response_json)
        return response_json['label']

      else:
        error_prompt = f"Error message: {error_message}. "
        prompt = llm.prompt_template(error_prompt, llm.prompt_template(system=system_prompt(), prompt=task))
        num_of_reprompts += 1

  mistral = Mistral(os.getenv('MISTRAL_API_KEY'))
  return extract_valid_json(output, mistral, json_schema)['label']


def extract_valid_json(invalid_json: str,
                       llm: BaseLLM,
                       json_format: dict,
                       model_name: str = "open-mistral-7b",
                       user: str = "",
                       max_retries: int = 7,
                       ) -> dict:
    """
    :return: JSON object containing the label and thoughts
    """

    print("***** Extracting Valid JSON ******")

    num_of_reprompts = 0
    sys = llm.prompt_template("system", json_sys())
    user = llm.prompt_template("user", invalid_json)
    llm.add_messages([sys, user])

    # HANDLE GENERATION & RETRIES
    for retry in range(max_retries):

        # If reached 3 consecutive retries, reset conversation context and start again
        if (retry + 1) % 3 == 0:
            llm.reset_messages()
            llm.add_messages([sys, user])
            model_name = "open-mixtral-8x7b"

        # Generate move, thoughts JSON using LLM
        output: str = llm.generate_text(model_name=model_name)
        response_json, error_message = validate_response(output, json_format)

        # If response is valid, extract and generate benchmark metrics and return response
        if response_json:
            print(response_json)
            return response_json

        # If error, reprompt with error-message
        error_prompt = f"Error message: {error_message}. "
        regenerate_prompt = f"Previous prompt: '''{user['content']}'''"
        llm.add_messages(
            [llm.prompt_template("assistant", output), llm.prompt_template("user", error_prompt + regenerate_prompt)])

        num_of_reprompts += 1

    return {'thoughts': 'Exceeded maximum retries', 'label': -1}


def json_sys():
    return """ You are an expert in natural language reasoning and inference. Your task is to analyze pairs of sentences and determine if the second sentence (hypothesis) can be logically inferred from the first sentence (premise). For each example, I will provide the premise and hypothesis. Your response should be in the following JSON format:
    {
      "thought_process":
        "Step 1. <Identify key information and relationships in the premise, considering logical connections, commonsense understanding, and factual consistency>.
        Step 2. <Analyze how the hypothesis relates to or contradicts the premise based on the information identified in Step 1. Evaluate if the hypothesis can be reasonably inferred from the premise>.
        Step 3. <Explain your final reasoning and conclusion on whether the hypothesis is entailed by the premise or not>",
      "label": "<0 for no entailment, 1 for entailment>"
    }
    Please provide a clear multi-step reasoning chain explaining how you arrived at your final answer, breaking it down into logical components. Ground your response in the given information, logical principles and common-sense reasoning.

    Example:

    Premise: The dog chased the cat up the tree. Hypothesis: The cat climbed the tree. Label:

    {
        "thought_process": "
            Step 1: the premise indicates a scenario where a dog chases a cat, resulting in the cat moving up a tree. The movement 'up the tree' suggests a vertical ascent, typical of climbing behavior. It is common sense that a cat would climb a tree to escape a chasing dog, and there are no known facts that contradict the premise or hypothesis.
            Step 2: 'The cat climbed the tree' can be logically inferred from the premise because the action of climbing is a reasonable and necessary part of the cat moving 'up the tree' as described. Thus, the hypothesis logically follows from the premise.
            Step 3: Based on the logical reasoning, common sense, and lack of contradictory facts, the hypothesis can be inferred from the premise.
            ",
        "label": 1
    }
    """

In [80]:
path_to_dataset = 'data/test_data/test.csv'
test = pd.read_csv(root + path_to_dataset)
test

,premise,hypothesis
0,"Boy wearing red hat, blue jacket pushing plow ...",The boy is surrounded by snow
1,A blond woman in a black shirt is standing beh...,The woman is standing.
2,Three people in uniform are outdoors and are o...,Uniformed people are outside
3,"A person, in a striped blue shirt and pants, i...",The person is running
4,"A man, woman, and child get their picture take...",A family on vacation is posing.
...,...,...
3297,A younger man dressed in tribal attire.,A young man wears clothing.
3298,A woman within an orchestra is playing a violin.,A man is looking in a telescope.
3299,You can see the bottom halves of three boys st...,There are three boys standing.
3300,A crowd of people fill the street in front of ...,The crowd is playing hockey.


In [81]:
sample = test.iloc[:5]
sample['prediction'] = sample.apply(lambda x : predict_label(x['premise'], x['hypothesis'], mistral, json_schema), axis=1)
sample

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


****** INPUT ******
[INST]  You are an expert in natural language reasoning and inference. Your task is to analyze pairs of sentences and determine if the second sentence (hypothesis) can be logically inferred from the first sentence (premise). For each example, I will provide the premise and hypothesis. Your response should be in the following JSON format:
    {
      "thought_process":
        "Step 1. <Identify key information and relationships in the premise, considering logical connections, commonsense understanding, and factual consistency>.
        Step 2. <Analyze how the hypothesis relates to or contradicts the premise based on the information identified in Step 1. Evaluate if the hypothesis can be reasonably inferred from the premise>.
        Step 3. <Explain your final reasoning and conclusion on whether the hypothesis is entailed by the premise or not>",
      "label": "<0 for no entailment, 1 for entailment>"
    }
    Please provide a clear multi-step reasoning chain exp

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a scenario where a boy is wearing a red hat and a blue jacket, pushing a plow in the snow. The presence of snow is explicitly mentioned in the premise.
        Step 2: The hypothesis 'The boy is surrounded by snow' can be logically inferred from the premise because the presence of snow is explicitly mentioned in the premise. The boy is pushing a plow in the snow, which implies that he is surrounded by snow.
        Step 3: Based on the logical reasoning and the explicit mention of snow in the premise, the hypothesis can be inferred from the premise.
        ",
    "label": 1
}

[/SOL]


****** SUCCESS ******
{'thought_process': "        step 1: the premise describes a scenario where a boy is wearing a red hat and a blue jacket, pushing a plow in the snow. the presence of snow is explicitly mentioned in the premise.        step 2: the hypothesis 'the boy is surrounded by snow' can be logically inferred

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a scenario where three people in uniform are outdoors and observing a scene that is out of the picture. The uniforms suggest that the people are part of a uniformed organization, such as police or military. The outdoor setting implies that the people are outside of a building or other enclosed space. The observation of a scene that is out of the picture suggests that the people are not directly involved in the scene but are witnessing it from a distance.
        Step 2: The hypothesis 'Uniformed people are outside' can be logically inferred from the premise because the premise provides sufficient information to conclude that the people in uniform are outside. The uniforms, outdoor setting, and observation of a scene that is out of the picture all support the conclusion that the people are uniformed and outside.
        Step 3: Based on the logical reasoning and lack of contradictory information, the h

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a person in a striped blue shirt and pants running along. The verb 'running' indicates a continuous action, and the adverb 'along' suggests a direction.
        Step 2: The hypothesis 'The person is running' is a logical conclusion from the premise because the action of running is a continuous and directional movement, which is consistent with the description in the premise.
        Step 3: Based on the logical reasoning and the lack of contradictory information, the hypothesis can be inferred from the premise.
        ",
    "label": 1
}

[/SOL]


****** SUCCESS ******
{'thought_process': "        step 1: the premise describes a person in a striped blue shirt and pants running along. the verb 'running' indicates a continuous action, and the adverb 'along' suggests a direction.        step 2: the hypothesis 'the person is running' is a logical conclusion from the premise because the action of running 

<ipython-input-81-cbb3d98dd97d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['prediction_json'] = sample.apply(lambda x : predict_label(x['premise'], x['hypothesis'], mistral, json_schema), axis=1)


,premise,hypothesis,prediction_json
0,"Boy wearing red hat, blue jacket pushing plow ...",The boy is surrounded by snow,1
1,A blond woman in a black shirt is standing beh...,The woman is standing.,0
2,Three people in uniform are outdoors and are o...,Uniformed people are outside,1
3,"A person, in a striped blue shirt and pants, i...",The person is running,1
4,"A man, woman, and child get their picture take...",A family on vacation is posing.,1


In [ ]:
sample = test.iloc[:1000]
sample['prediction_json'] = sample.apply(lambda x : predict_label(x['premise'], x['hypothesis'], mistral, json_schema), axis=1)
sample.to_csv(root + f'data/test_data/results_{sample.shape[0]}.csv')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


****** INPUT ******
[INST]  You are an expert in natural language reasoning and inference. Your task is to analyze pairs of sentences and determine if the second sentence (hypothesis) can be logically inferred from the first sentence (premise). For each example, I will provide the premise and hypothesis. Your response should be in the following JSON format:
    {
      "thought_process":
        "Step 1. <Identify key information and relationships in the premise, considering logical connections, commonsense understanding, and factual consistency>.
        Step 2. <Analyze how the hypothesis relates to or contradicts the premise based on the information identified in Step 1. Evaluate if the hypothesis can be reasonably inferred from the premise>.
        Step 3. <Explain your final reasoning and conclusion on whether the hypothesis is entailed by the premise or not>",
      "label": "<0 for no entailment, 1 for entailment>"
    }
    Please provide a clear multi-step reasoning chain exp

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a scenario where a boy is wearing a red hat and a blue jacket, pushing a plow in the snow. The presence of snow is explicitly mentioned in the premise.
        Step 2: The hypothesis 'The boy is surrounded by snow' can be logically inferred from the premise because the presence of snow is explicitly mentioned in the premise. The boy is pushing a plow in the snow, which implies that he is surrounded by snow.
        Step 3: Based on the logical reasoning and the explicit mention of snow in the premise, the hypothesis can be inferred from the premise.
        ",
    "label": 1
}

[/SOL]


****** SUCCESS ******
{'thought_process': "        step 1: the premise describes a scenario where a boy is wearing a red hat and a blue jacket, pushing a plow in the snow. the presence of snow is explicitly mentioned in the premise.        step 2: the hypothesis 'the boy is surrounded by snow' can be logically inferred

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a scenario where two white dogs are fighting with each other. This implies a physical confrontation between the dogs, likely involving biting, growling, and other aggressive behaviors.
        Step 2: The hypothesis 'The two dogs are having a brawl' is a logical inference from the premise. The term 'brawl' refers to a violent and intense fight, which is consistent with the actions described in the premise. The premise provides sufficient information to support the conclusion that the dogs are engaged in a brawl.
        Step 3: Based on the logical reasoning and the information provided in the premise, the hypothesis can be inferred from the premise.
        ",
    "label": 1
}

[/SOL]


****** SUCCESS ******
{'thought_process': "        step 1: the premise describes a scenario where two white dogs are fighting with each other. this implies a physical confrontation between the dogs, likely involving b

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a scenario where a group of people are examining a boat with an orange flag, which is sitting on sand next to a body of water. The boat and flag are not mentioned in the hypothesis.
        Step 2: The hypothesis 'People don't like sand' is not logically entailed by the premise. The premise does not provide any information about the people's preferences or opinions regarding sand.
        Step 3: Based on the logical reasoning and lack of contradictory information, the hypothesis is not entailed by the premise.
        ",
    "label": 0
}

[/SOL]


****** SUCCESS ******
{'thought_process': "        step 1: the premise describes a scenario where a group of people are examining a boat with an orange flag, which is sitting on sand next to a body of water. the boat and flag are not mentioned in the hypothesis.        step 2: the hypothesis 'people don't like sand' is not logically entailed by the premise.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***** OUTPUT *****


{
    "thought_process": "
        Step 1: The premise describes a specific scenario involving a swimmer performing the breaststroke in an indoor swimming pool. The context is clear and there are no contradictory facts.
        Step 2: The hypothesis describes a different scenario involving a man riding a bike. There is no logical connection between the premise and the hypothesis. The premise does not provide any information about a man riding a bike.
        Step 3: Based on the logical reasoning, common sense, and lack of contradictory facts, the hypothesis cannot be inferred from the premise.
        ",
    "label": 0
}

[/SOL]


****** SUCCESS ******
{'thought_process': '        step 1: the premise describes a specific scenario involving a swimmer performing the breaststroke in an indoor swimming pool. the context is clear and there are no contradictory facts.        step 2: the hypothesis describes a different scenario involving a man riding a bike. there is n

In [12]:
import json

# Open the JSON file
with open('prompt_engineering/test.json') as json_file:
    # Load the JSON data
    data = json.load(json_file)
pd.DataFrame(data).to_csv('data/test_data/predictions_0_329.csv')

In [16]:
test = pd.read_csv('data/test_data/test.csv').iloc[0:329]

In [17]:
predictions = pd.read_csv('data/test_data/predictions_0_329.csv').drop_duplicates(subset='thought_process')

In [29]:
def print_row(row):
    h = row['hypothesis']
    p = row['premise']
    t = row['thought_process']
    l = row['label']
    
    print(f"h: {h}\np: {p}\nt: {t}\nl: {l}")

all = test.join(predictions)

In [30]:
all.apply(print_row, axis=1)

h: The boy is surrounded by snow
p: Boy wearing red hat, blue jacket pushing plow in snow.
t:         step 1: the premise describes a scenario where a boy is wearing a red hat and a blue jacket, pushing a plow in the snow. the presence of snow is explicitly mentioned in the premise.        step 2: the hypothesis 'the boy is surrounded by snow' can be logically inferred from the premise because the presence of snow is explicitly mentioned in the premise. the boy is pushing a plow in the snow, which implies that he is surrounded by snow.        step 3: based on the logical reasoning and the explicit mention of snow in the premise, the hypothesis can be inferred from the premise.        
l: 1
h: The woman is standing.
p: A blond woman in a black shirt is standing behind a counter.
t:         step 1: the premise describes a woman in a black shirt standing behind a counter.        step 2: the hypothesis is a logical conclusion from the premise, as it is a reasonable and necessary part of th

0      None
1      None
2      None
3      None
4      None
       ... 
324    None
325    None
326    None
327    None
328    None
Length: 329, dtype: object

In [ ]:
import pandas as pd

path_to_dataset = 'data/test_data/test.csv' # Replace with path to new data
test = pd.read_csv(root + path_to_dataset)
test['id'] = test.index

for i in range(0, 35):
    if i == 3: continue
    lo, hi = i * 100, (i+1) * 100
    test_sample = test.iloc[lo:hi]
    test_sample['prediction'] = test_sample.apply(lambda x : predict_label(x['id'], x['premise'], x['hypothesis'], mistral, json_schema), axis=1)
    test_sample.to_csv(f'data/test_data/predictions_{lo}_{hi}.csv')

import pandas as pd

path_to_dataset = 'data/training_data/dev.csv' # Replace with path to new data
dev = pd.read_csv(root + path_to_dataset)
dev['id'] = dev.index
dev_sample = test.sample(n=500)

for i in range(0, 5):
    if i == 3: continue
    lo, hi = i * 100, (i+1) * 100
    dev_sample = dev.iloc[lo:hi]
    dev_sample['prediction'] = dev.apply(lambda x : predict_label(x['id'], x['premise'], x['hypothesis'], mistral, json_schema), axis=1)
    dev_sample.to_csv(f'data/training_data/dev_predictions_{lo}_{hi}.csv')